In [7]:
import pandas as pd
import numpy as np

# =============================
# CONFIG
# =============================
PRES_FILE = "Presupuesto_Q4_2025.xlsx"
REAL_FILE = "Real_Q4_2025.xlsx"
OUTPUT_XLSX = r"C:\Users\agust\Downloads\Desvios_Q4_2025.xlsx"

TOP_CUENTAS = 5
TOP_DESVIOS = 5


# =============================
# HELPERS (FORMATO ARG)
# =============================
def money_ar(x):
    if pd.isna(x):
        return "-"
    return f"$ {int(round(x)):,}".replace(",", ".")


def pct_ar(x, decimals=1):
    if pd.isna(x) or x is None:
        return "-"
    return f"{round(x * 100, decimals):.{decimals}f}%".replace(".", ",")


def safe_div(a, b):
    return np.nan if b == 0 else a / b


# =============================
# MAIN
# =============================
def main():
    pres = pd.read_excel(PRES_FILE, sheet_name="Presupuesto")
    real = pd.read_excel(REAL_FILE, sheet_name="Real")

    # Normalización
    pres["Mes"] = pres["Mes"].astype(str).str[:7]
    real["Mes"] = real["Mes"].astype(str).str[:7]

    pres["Presupuesto"] = pd.to_numeric(pres["Presupuesto"], errors="coerce").fillna(0)
    real["Monto"] = pd.to_numeric(real["Monto"], errors="coerce").fillna(0)

    # Consolidar Real
    real_agg = (
        real.groupby(["Mes", "CentroCosto", "Cuenta"], as_index=False)
            .agg(Real=("Monto", "sum"))
    )

    # Merge
    df = pres.merge(real_agg, on=["Mes", "CentroCosto", "Cuenta"], how="left")
    df["Real"] = df["Real"].fillna(0)

    # Desvíos
    df["Desvio_ARS"] = df["Real"] - df["Presupuesto"]
    df["Desvio_Pct"] = np.where(
        df["Presupuesto"] == 0,
        np.nan,
        df["Desvio_ARS"] / df["Presupuesto"]
    )

    df["Interpretacion"] = np.where(
        df["Desvio_ARS"] > 0,
        "Desfavorable (sobregasto)",
        "Favorable (por debajo)"
    )

    df["ImpactoAbs"] = df["Desvio_ARS"].abs()

    # =============================
    # 📊 RESUMEN EJECUTIVO
    # =============================
    total_pres = df["Presupuesto"].sum()
    total_real = df["Real"].sum()
    total_desv = df["Desvio_ARS"].sum()

    ejec_total = safe_div(total_real, total_pres)
    desv_pct_total = safe_div(total_desv, total_pres)

    print("\n📊 RESUMEN CONTROL DE GESTIÓN - Q4 2025\n")

    print(f"Presupuesto total Q4 : {money_ar(total_pres)}")
    print(f"Real total Q4        : {money_ar(total_real)}")
    print(f"Ejecución Q4         : {pct_ar(ejec_total)}")
    print(f"Desvío total Q4      : {money_ar(total_desv)} ({pct_ar(desv_pct_total)})")

    # =============================
    # 📅 RESUMEN POR MES
    # =============================
    resumen_mes = (
        df.groupby("Mes", as_index=False)
          .agg(
              Presupuesto=("Presupuesto", "sum"),
              Real=("Real", "sum"),
              Desvio_ARS=("Desvio_ARS", "sum")
          )
    )
    resumen_mes["Ejecucion_%"] = resumen_mes.apply(lambda r: safe_div(r["Real"], r["Presupuesto"]), axis=1)
    resumen_mes["Desvio_%"] = resumen_mes.apply(lambda r: safe_div(r["Desvio_ARS"], r["Presupuesto"]), axis=1)

    print("\n📅 Resumen por mes")
    for _, r in resumen_mes.sort_values("Mes").iterrows():
        print(
            f"{r['Mes']} | Pres: {money_ar(r['Presupuesto'])} | "
            f"Real: {money_ar(r['Real'])} | "
            f"Ejec: {pct_ar(r['Ejecucion_%'])} | "
            f"Desvío: {money_ar(r['Desvio_ARS'])} ({pct_ar(r['Desvio_%'])})"
        )

    # =============================
    # 🧠 INSIGHT CLAVE
    # =============================
    print(
        "\n🧠 El desvío del trimestre se explica principalmente "
        "por la sobre-ejecución en Diciembre, impulsada por Sueldos y Marketing."
    )

    # =============================
    # 🧾 TOP CUENTAS
    # =============================
    cuentas = (
        df.groupby("Cuenta", as_index=False)
          .agg(
              Presupuesto=("Presupuesto", "sum"),
              Real=("Real", "sum"),
              Desvio_ARS=("Desvio_ARS", "sum")
          )
    )
    cuentas["ImpactoAbs"] = cuentas["Desvio_ARS"].abs()
    cuentas["Ejecucion_%"] = cuentas.apply(lambda r: safe_div(r["Real"], r["Presupuesto"]), axis=1)

    top_cuentas = (
        cuentas.sort_values("ImpactoAbs", ascending=False)
               .head(TOP_CUENTAS)
               .drop(columns="ImpactoAbs")
    )

    print(f"\n🧾 Top {TOP_CUENTAS} cuentas que explican el desvío\n")
    show_c = top_cuentas.copy()
    show_c["Presupuesto"] = show_c["Presupuesto"].map(money_ar)
    show_c["Real"] = show_c["Real"].map(money_ar)
    show_c["Desvio_ARS"] = show_c["Desvio_ARS"].map(money_ar)
    show_c["Ejecucion_%"] = show_c["Ejecucion_%"].map(pct_ar)
    print(show_c.to_string(index=False))

    # =============================
    # 🔎 TOP DESVÍOS RELEVANTES
    # =============================
    top = (
        df.sort_values("ImpactoAbs", ascending=False)
          .head(TOP_DESVIOS)
          .loc[:, ["Mes", "CentroCosto", "Cuenta", "Presupuesto", "Real", "Desvio_ARS", "Interpretacion"]]
    )

    print(f"\n🔎 Top {TOP_DESVIOS} desvíos más relevantes)\n")
    top_show = top.copy()
    top_show["Presupuesto"] = top_show["Presupuesto"].map(money_ar)
    top_show["Real"] = top_show["Real"].map(money_ar)
    top_show["Desvio_ARS"] = top_show["Desvio_ARS"].map(money_ar)
    print(top_show.to_string(index=False))

    # =============================
    # EXPORT A EXCEL
    # =============================
    detalle = (
        df.drop(columns=["ImpactoAbs"])
          .sort_values(["Mes", "CentroCosto", "Cuenta"])
          .reset_index(drop=True)
          .rename(columns={"Desvio_ARS": "Desvio_$", "Desvio_Pct": "Desvio_%"})
    )

    resumen_excel = resumen_mes.rename(columns={"Desvio_ARS": "Desvio_$"})
    top_excel = top.rename(columns={"Desvio_ARS": "Desvio_$"})
    top_cuentas_excel = top_cuentas.rename(columns={"Desvio_ARS": "Desvio_$"})

    resumen_totales = pd.DataFrame([{
        "Mes": "Q4_TOTAL",
        "Presupuesto": total_pres,
        "Real": total_real,
        "Desvio_$": total_desv,
        "Ejecucion_%": ejec_total,
        "Desvio_%": desv_pct_total
    }])

    resumen_excel = pd.concat([resumen_excel, resumen_totales], ignore_index=True)

    with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl") as writer:
        resumen_excel.to_excel(writer, index=False, sheet_name="Resumen")
        top_cuentas_excel.to_excel(writer, index=False, sheet_name="Top_Cuentas")
        top_excel.to_excel(writer, index=False, sheet_name="Top_Desvios")
        detalle.to_excel(writer, index=False, sheet_name="Detalle")

    print(f"\n✅ Excel generado correctamente en:\n{OUTPUT_XLSX}")


if __name__ == "__main__":
    main()



📊 RESUMEN CONTROL DE GESTIÓN - Q4 2025

Presupuesto total Q4 : $ 166.166.885
Real total Q4        : $ 168.786.414
Ejecución Q4         : 101,6%
Desvío total Q4      : $ 2.619.529 (1,6%)

📅 Resumen por mes
2025-10 | Pres: $ 53.655.261 | Real: $ 53.411.297 | Ejec: 99,5% | Desvío: $ -243.964 (-0,5%)
2025-11 | Pres: $ 55.258.397 | Real: $ 54.798.129 | Ejec: 99,2% | Desvío: $ -460.268 (-0,8%)
2025-12 | Pres: $ 57.253.227 | Real: $ 60.576.988 | Ejec: 105,8% | Desvío: $ 3.323.761 (5,8%)

🧠 El desvío del trimestre se explica principalmente por la sobre-ejecución en Diciembre, impulsada por Sueldos y Marketing.

🧾 Top 5 cuentas que explican el desvío

   Cuenta  Presupuesto         Real  Desvio_ARS Ejecucion_%
  Sueldos $ 76.033.142 $ 77.181.459 $ 1.148.317      101,5%
Marketing $ 22.182.166 $ 23.251.354 $ 1.069.188      104,8%
       IT  $ 9.916.362 $ 10.345.714   $ 429.352      104,3%
Logística $ 10.990.272 $ 11.338.065   $ 347.793      103,2%
Servicios  $ 9.153.773  $ 8.871.017  $ -282.756 